In [14]:
import pandas as pd
import numpy as np

import librosa

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.math import confusion_matrix
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

from tqdm import tqdm

2022-03-02 17:50:33.067762: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-02 17:50:33.067846: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [15]:
def load_data(n):
    targets = pd.read_csv('targets.csv')
    group = targets.groupby('gender_emotion')
    sample = group.head(n)
    return sample

In [16]:
def cut_or_pad(array, n):
    if array.shape[1] < n:
        return pad_sequences(array,n,padding='post',value=-1000.)
    elif array.shape[1] > n:
        return array[:,:n]
    else:
        return array

In [17]:
def extract_features(df,
                     sr=44100,
                     length=250,
                     offset=0.3,
                     n_mfcc=10,
                     poly_order=4,
                     n_chroma=10,
                     n_mels=10):
    
    features = []
    
    for path in tqdm(df['path']):
        wav = librosa.load(path, sr=sr, offset=offset, duration = 10)
        
        mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
        mfcc = cut_or_pad(mfcc,length)
        
        poly = librosa.feature.poly_features(y=wav[0],sr=sr,order=poly_order)
        poly = cut_or_pad(poly,length)
        
        chroma = librosa.feature.chroma_stft(y=wav[0],sr=sr,n_chroma=n_chroma)
        chroma = cut_or_pad(chroma,length)
    
        melspec = librosa.feature.melspectrogram(y=wav[0],sr=sr,n_mels=n_mels)
        melspec = cut_or_pad(melspec,length)
        log_S = librosa.amplitude_to_db(melspec)
    
#         zcr = librosa.feature.zero_crossing_rate(y=wav[0])
#         zcr = cut_or_pad(zcr,length)
        
#         rms = librosa.feature.rms(y=wav[0])
#         rms = cut_or_pad(rms,length)
    
    #     f_tempo = librosa.feature.fourier_tempogram(y=wav[0], sr=sr, win_length=18)
    #     f_tempo = cut_or_pad(f_tempo,length)
        
    #     tempo = librosa.feature.tempogram(y=wav[0], sr=SAMPLE_RATE, win_length=10)
    #     tempo = cut_or_pad(tempo,length)
        
    #     spectral_centroid = librosa.feature.spectral_centroid(y=wav[0],sr=sr)
    #     spectral_centroid = cut_or_pad(spectral_centroid,length)
        
    #     spectral_bandwidth = librosa.feature.spectral_bandwidth(y=wav[0],sr=sr)
    #     spectral_bandwidth = cut_or_pad(spectral_bandwidth,length)
        
    #     spectral_contrast = librosa.feature.spectral_contrast(y=wav[0],sr=sr)
    #     spectral_contrast = cut_or_pad(spectral_contrast,length)
        
    #     spectral_flatness = librosa.feature.spectral_flatness(y=wav[0])
    #     spectral_flatness = cut_or_pad(spectral_flatness,length)
        
    #     spectral_rolloff = librosa.feature.spectral_rolloff(y=wav[0])
    #     spectral_rolloff = cut_or_pad(spectral_rolloff,length)
        
    #     tonnetz = librosa.feature.tonnetz(y=wav[0],sr=sr)
    #     tonnetz = cut_or_pad(tonnetz,length)
        
    #     chroma_cqt = librosa.feature.chroma_cqt(y=wav[0],sr=sr,n_chroma=10,bins_per_octave=30)
    #     chroma_cqt = cut_or_pad(chroma_cqt,length)
        
    #     chroma_cens = librosa.feature.chroma_cens(y=wav[0],sr=sr,n_chroma=10,bins_per_octave=30)
    #     chroma_cens = cut_or_pad(chroma_cens,length)
    
    
        #combine
        array = np.vstack((mfcc,
                           poly,
                           chroma,
                           log_S,
#                            zcr,
#                            rms,
    #                        f_tempo,
    #                        tempo,
    #                        spectral_centroid,
    #                        spectral_bandwidth,
    #                        spectral_contrast ,
    #                        spectral_flatness ,
    #                        spectral_rolloff  ,
    #                        tonnetz           ,
    #                        chroma_cqt        ,
    #                        chroma_cens,
                      ))
    
        array = array.T

        #mfcc = np.concatenate((mfcc,poly),axis=0,dtype=None)
        features.append(array)
    
    X = pad_sequences(features, dtype='float32', padding='post', value=-1000.)
    return X

In [18]:
def Xy(df,array, test_size=0.2):
    y = df['emotion']
    le = LabelEncoder()
    y_num = le.fit_transform(y)
    y_cat = to_categorical(y_num)
    
    X_train, X_val, y_train, y_val = train_test_split(array, y_cat, test_size=test_size, random_state=0)
    
    return X_train, X_val, y_train, y_val

In [19]:
def make_model(X):
    model = Sequential()
    model.add(layers.Masking(mask_value = -1000., input_shape=(X.shape[1], X.shape[2])))
    model.add(layers.LSTM(units=X_train.shape[2], return_sequences=True, activation='tanh'))
    model.add(layers.LSTM(64, return_sequences=False, activation='tanh'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(20, activation='relu'))
    #model.add(layers.Dropout(0.1))
    model.add(layers.Dense(15, activation='relu'))
    #model.add(layers.Dropout(0.1))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(6, activation="softmax"))
    
    optimiser = Adam(learning_rate=0.003)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=optimiser,metrics='acc')
    
    model.summary()
    return model

In [20]:
data = load_data(150)

In [15]:
X = extract_features(data,
                     sr=44100,
                     length=260,
                     offset=0.2,
                     n_mfcc=12,
                     poly_order=3,
                     n_chroma=10,
                     n_mels=10)

  0%|                                                                                          | 0/1800 [00:00<?, ?it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[-6.6712564e-03 -7.6333415e-03 -7.3763202e-03 ...  9.9765680e-07
  3.1930197e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  0%|                                                                                  | 1/1800 [00:00<19:27,  1.54it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[ 2.6515196e-03  3.0636524e-03  2.9433763e-03 ...  6.4893459e-07
 -2.2089328e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  0%|                                                                                  | 2/1800 [00:00<11:51,  2.53it/s]/tmp/ipykern

  2%|█▊                                                                               | 39/1800 [00:08<06:03,  4.85it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[-1.4199197e-02 -1.6325003e-02 -1.6029138e-02 ... -5.6605835e-08
  1.3802916e-07  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  2%|█▊                                                                               | 40/1800 [00:08<06:10,  4.75it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[0.00412379 0.00490092 0.00508746 ... 0.00023626 0.00015272 0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  2%|█▊                                                                               | 41/1800 [00:08<06:26,  4.55it/s]/tmp/ipykernel_24344/980438913.py:15:

  4%|███▍                                                                             | 77/1800 [00:16<06:11,  4.64it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[-2.8098836e-03 -3.0663570e-03 -2.9042100e-03 ...  1.2984621e-08
  7.5259088e-10  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  4%|███▌                                                                             | 78/1800 [00:16<06:12,  4.62it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[ 4.3090782e-03  5.0142305e-03  4.9758344e-03 ... -5.8791520e-06
  1.6771075e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  4%|███▌                                                                             | 79/1800 [00:17<05:59,  4.79it/s]/tmp/ipykern

  6%|█████▏                                                                          | 116/1800 [00:27<11:21,  2.47it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[-2.15848582e-03 -2.70173280e-03 -2.82212254e-03 ...  1.09395245e-04
  7.08680309e-05  0.00000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  6%|█████▏                                                                          | 117/1800 [00:27<10:36,  2.65it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[0.00300602 0.00357936 0.00376153 ... 0.         0.         0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  7%|█████▏                                                                          | 118/1800 [00:27<09:58,  2.81it/s]/tmp/ipykernel_24344/980438913.

  9%|██████▉                                                                         | 155/1800 [00:37<06:54,  3.97it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[-4.2556989e-05 -2.2863296e-05  8.7119966e-05 ...  4.3552648e-10
  5.7265614e-10  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  9%|██████▉                                                                         | 156/1800 [00:38<06:56,  3.94it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[-0.00250298 -0.00281068 -0.00282655 ...  0.00076661  0.00050516
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
  9%|██████▉                                                                         | 157/1800 [00:38<06:48,  4.02it/s]/tmp/ipykernel_24344/980438913

...

 91%|███████████████████████████████████████████████████████████████████████▉       | 1638/1800 [06:46<00:32,  5.01it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[ 2.4908720e-02  2.7763680e-02  2.5591457e-02 ... -4.5729143e-05
 -4.0502513e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
 91%|███████████████████████████████████████████████████████████████████████▉       | 1639/1800 [06:46<00:33,  4.86it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[-1.0113510e-03 -1.1807693e-03 -1.1466966e-03 ... -6.3205400e-05
 -2.9680905e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
 91%|███████████████████████████████████████████████████████████████████████▉       | 1640/1800 [06:47<00:32,  4.98it/s]/tmp/ipykern

 93%|█████████████████████████████████████████████████████████████████████████▌     | 1676/1800 [06:54<00:26,  4.77it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[-0.00554544 -0.0064487  -0.00642685 ...  0.00124664  0.00078977
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
 93%|█████████████████████████████████████████████████████████████████████████▌     | 1677/1800 [06:54<00:26,  4.72it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[0.00362788 0.00278662 0.00072093 ... 0.         0.         0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
 93%|█████████████████████████████████████████████████████████████████████████▋     | 1678/1800 [06:54<00:25,  4.87it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pa

 95%|███████████████████████████████████████████████████████████████████████████▏   | 1714/1800 [07:02<00:18,  4.77it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[ 2.1762331e-03  2.1796154e-03  1.7928369e-03 ... -2.8292288e-13
  6.3371015e-13  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
 95%|███████████████████████████████████████████████████████████████████████████▎   | 1715/1800 [07:02<00:17,  4.80it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[ 8.5034976e-03  9.8844729e-03  9.7573008e-03 ... -3.5538559e-07
 -5.4878467e-07  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
 95%|███████████████████████████████████████████████████████████████████████████▎   | 1716/1800 [07:02<00:17,  4.81it/s]/tmp/ipykern

 97%|████████████████████████████████████████████████████████████████████████████▉  | 1753/1800 [07:10<00:10,  4.35it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[9.66821425e-03 1.11073665e-02 1.08039482e-02 ... 2.95142381e-05
 2.51097990e-05 0.00000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
 97%|████████████████████████████████████████████████████████████████████████████▉  | 1754/1800 [07:10<00:10,  4.52it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[-1.2438963e-03 -1.7027893e-03 -1.8474380e-03 ...  1.1352606e-04
  7.3389827e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
 98%|█████████████████████████████████████████████████████████████████████████████  | 1755/1800 [07:11<00:10,  4.44it/s]/tmp/ipykern

100%|██████████████████████████████████████████████████████████████████████████████▌| 1791/1800 [07:20<00:02,  4.17it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[-2.8416563e-03 -3.7063626e-03 -4.1509666e-03 ... -1.8629935e-07
 -1.8683876e-07  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
100%|██████████████████████████████████████████████████████████████████████████████▋| 1792/1800 [07:20<00:02,  3.87it/s]/tmp/ipykernel_24344/980438913.py:15: FutureWarning: Pass y=[-2.0119151e-02 -2.5079010e-02 -2.7214086e-02 ... -8.1944854e-05
 -5.7773515e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wav[0], sr=sr, n_mfcc=n_mfcc)
100%|██████████████████████████████████████████████████████████████████████████████▋| 1793/1800 [07:20<00:01,  3.65it/s]/tmp/ipykern

In [17]:
X_train, X_val, y_train, y_val = Xy(data,X)

In [18]:
model = make_model(X_train)

2022-03-01 20:12:16.446010: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-01 20:12:16.446259: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-01 20:12:16.446462: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-LF7KQEFR): /proc/driver/nvidia/version does not exist
2022-03-01 20:12:16.448183: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 260, 36)           0         
                                                                 
 lstm (LSTM)                 (None, 260, 36)           10512     
                                                                 
 lstm_1 (LSTM)               (None, 64)                25856     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 20)                660       
                                                                 
 dense_2 (Dense)             (None, 15)                315       
                                                                 
 dense_3 (Dense)             (None, 10)                1

In [28]:
es = EarlyStopping(patience=20,monitor='val_loss',restore_best_weights=True)

model.fit(X_train, y_train, validation_data=(X_val,y_val), callbacks=es,
         batch_size=32,
         epochs=100, verbose=1)

Epoch 1/100


2022-03-01 19:30:09.265764: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 80870400 exceeds 10% of free system memory.


34/34 [==============================] - 22s 640ms/step - loss: 1.7517 - acc: 0.2046 - val_loss: 1.7120 - val_acc: 0.2519
Epoch 2/100
34/34 [==============================] - 18s 525ms/step - loss: 1.7172 - acc: 0.2444 - val_loss: 1.6924 - val_acc: 0.3185
Epoch 3/100
34/34 [==============================] - 18s 524ms/step - loss: 1.6749 - acc: 0.2722 - val_loss: 1.6777 - val_acc: 0.3019
Epoch 4/100
34/34 [==============================] - 18s 523ms/step - loss: 1.6662 - acc: 0.2787 - val_loss: 1.6674 - val_acc: 0.2926
Epoch 5/100
34/34 [==============================] - 18s 523ms/step - loss: 1.6952 - acc: 0.2588 - val_loss: 1.7129 - val_acc: 0.2648
Epoch 6/100
34/34 [==============================] - 18s 518ms/step - loss: 1.6810 - acc: 0.2667 - val_loss: 1.7376 - val_acc: 0.2074
Epoch 7/100
34/34 [==============================] - 18s 528ms/step - loss: 1.7278 - acc: 0.2153 - val_loss: 1.7632 - val_acc: 0.2389
Epoch 8/100
34/34 [==============================] - 18s 528ms/step - loss

KeyboardInterrupt: 

In [19]:
pip install speechbrain

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from speechbrain.pretrained import EncoderClassifier

In [17]:
from speechbrain.pretrained.interfaces import foreign_class

In [20]:
model = foreign_class(source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP",
                      pymodule_file="custom_interface.py",
                      classname="CustomEncoderWav2vec2Classifier")


Please install transformer from HuggingFace to use wav2vec2/Hubert !


ErrorDuringImport: problem in speechbrain.lobes.models.huggingface_wav2vec - NameError: name 'Wav2Vec2Model' is not defined